In [162]:
#import required packages
import pandas as pd
import numpy as np
import holoviews as hv
import hvplot.pandas
import panel as pn
import plotly.express as px
import scipy.stats as sp

In [ ]:
# Read processed data
scot_mfg_df = pd.read_pickle('interim/scot_adv_mfg_df.pkl')
scot_mfg_df.rename(columns = {"TotalDealroomFundingMillionEuros":"DealroomFunding"}, inplace = True)
scot_mfg_df['DealroomFunding'] = scot_mfg_df['DealroomFunding'].replace(np.nan, 0)


In [ ]:
df_rtics = pd.DataFrame({})
cols =['Robotics and Automation','Forging','Forming','Computer Aided Manufacturing','Prototyping',
       'Photonics','Digital Twins','Digital Design','Mixed Reality','Renewables']
for col in cols:

    data = scot_mfg_df["RTICsubsectors"].str.contains(col)

    df = scot_mfg_df[data]

    df = scot_mfg_df[data].where(scot_mfg_df["BestEstimateEmployees"]>0)
    df = df.dropna(subset=['BestEstimateEmployees'])
    df.reset_index(drop=True, inplace=True)

    s = pd.Series(np.tile([col], len(df)))
    df['RTIC'] = s

    df['yearly growth rate'] = ((df.Turnover_2017-df.Turnover_2016)+(df.Turnover_2018-df.Turnover_2017)+(df.Turnover_2019-df.Turnover_2018)+(df.Turnover_2020-df.Turnover_2019))/df.BestEstimateTurnover    
    df_rtics = pd.concat([df_rtics, df])

df_rtics.reset_index(drop=True, inplace=True)

In [ ]:
process_df2 = df_rtics.mask(df_rtics["BestEstimateGrowthPercentagePerYear"]==0)
process_df2['Incorporation Year']= pd.to_datetime(df_rtics.IncorporationDate).dt.year
process_df2.rename(columns = {"BestEstimateGrowthPercentagePerYear":"Growth Per Year"}, inplace = True)
process_df2 = process_df2.dropna(subset=['Growth Per Year'])
process_df2.reset_index(drop=True, inplace=True)
df2 = process_df2.groupby('RTIC', group_keys=True).apply(lambda x: x)

def plot_scatter():
    
    fig_a = process_df2.hvplot(x='Incorporation Year', y='Growth Per Year', by='RTIC', kind='scatter', alpha=0.4, persist=True
                       ,ylim=[-1,1],width = 1200,height = 400, hover_cols=['Companyname', 'RTICsubsectors'],grid =True)
    #df3=df2.sort_values(by='Incorporation Year', ascending = True)
    #df3.set_index('RTIC', drop=True, append=False, inplace=True)
    #fig_line = df3.hvplot(x='Incorporation Year', y='Growth Per Year')
    return fig_a

In [ ]:
def sub_plot_scatter():
    fig_b = process_df2.hvplot(x='Incorporation Year', y='Growth Per Year', by='RTIC', kind='scatter',
                   ylim=[-1,1],subplots=True,width = 300,height = 250, hover_cols=['Companyname', 'RTICsubsectors'],grid =True).cols(4)
    return fig_b

In [ ]:
# Plot total count for each sector and cumulative percentage
db_explanation_pane = pn.pane.Markdown("""
# Incorporation Year vs. growth per year
""", width=1200)

count_db_size_layout = pn.Column(db_explanation_pane, plot_scatter)
db_size_layout = pn.Column(count_db_size_layout, sub_plot_scatter)

In [ ]:
cols =['Robotics and Automation','Forging','Forming','Computer Aided Manufacturing','Prototyping',
       'Photonics','Digital Twins','Digital Design','Mixed Reality','Renewables']
RTIC_catg = pn.widgets.Select(name='Chose a signal', options=cols, value = 'Robotics and Automation')
df2 = process_df2.groupby('RTIC', group_keys=True).apply(lambda x: x)

@pn.depends (RTIC_catg.param.value)
def plot_select(RTIC_catg):

    comp_list = list(df2.loc[RTIC_catg,:].Companyname)
    multi_select = pn.widgets.MultiSelect(name='List of Companies', options=comp_list, size=8)
    
    return multi_select

multi_select = plot_select(RTIC_catg.value)
button = pn.widgets.Button(name="Update Plot", button_type="primary",width=400)


def make_fig():
    
    remove = multi_select.value
    process_df3 = process_df2.copy()
    process_df3.set_index('Companyname', drop=True, append=False, inplace=True)
    process_df3 = process_df3.drop(remove)
    process_df3.reset_index(drop=True, inplace=True)
    
    c3_df = pd.DataFrame({})
    c3_df['growth rate (mean)'] = process_df3.groupby('RTIC').agg({"Growth Per Year": "mean"})
    c3_df['growth rate (std)'] = process_df3.groupby('RTIC').agg({"Growth Per Year": np.std})
    #c3_df['growth rate (skew)'] = process_df3.groupby('RTIC').agg({"Growth Per Year": sp.skew})
    c3_df = c3_df.sort_values(by='growth rate (mean)', ascending = True)
    fig_c = c3_df.hvplot.bar(height=600, width=1200, rot=75)
    
    return fig_c
app = pn.Column(pn.Column(RTIC_catg,multi_select),button, make_fig)

def make_fig_2():
    
    remove = app[0][1].value
    process_df3 = process_df2.copy()
    process_df3.set_index('Companyname', drop=True, append=False, inplace=True)
    process_df3 = process_df3.drop(remove)
    process_df3.reset_index(drop=True, inplace=True)
    
    c3_df = pd.DataFrame({})
    c3_df['growth rate (mean)'] = process_df3.groupby('RTIC').agg({"Growth Per Year": "mean"})
    c3_df['growth rate (std)'] = process_df3.groupby('RTIC').agg({"Growth Per Year": np.std})
    #c3_df['growth rate (skew)'] = process_df3.groupby('RTIC').agg({"Growth Per Year": sp.skew})
    c3_df = c3_df.sort_values(by='growth rate (mean)', ascending = True)
    fig_c = c3_df.hvplot.bar(height=600, width=1200, rot=75)
    
    return fig_c

def update(event):
    app[0][1] = plot_select(RTIC_catg.value)
    
# update or replace the already rendered components when a watched parameter changes
RTIC_catg.param.watch(update, 'value')

app = pn.Column(pn.Column(RTIC_catg,multi_select),button, make_fig)

def b(event):

    app[2] = make_fig_2()
    
button.on_click(b)

In [ ]:
# Used defined colormap
s='''
    red, yellow, darkmagenta,orange,
    magenta,blue,
    greenyellow, cyan, indianred, indigo,
    darkgoldenrod, darkgreen, orange, darkolivegreen, darkorange,
    darkorchid, darkred, darksalmon, hotpink
    '''
colours = s.split(',')
colours = [c.replace('\n','') for c in colours]
colours = [c.replace(' ','') for c in colours]
colour_map = {rtics:c for rtics,c in zip(set(df_rtics.RTIC), colours)}
colour_map

In [ ]:
pn.extension('plotly')
#Plot estimate turnover of all the recorded companies w.r.t location
rev_geog_pane = pn.pane.Markdown("""
# Registered Location
""", width=1000)

rev_geog_fig = px.scatter_mapbox(df_rtics, lat="latitude", lon="longitude", color=df_rtics.RTIC, color_discrete_map=colour_map, 
                                     mapbox_style='open-street-map',hover_name='Companyname', hover_data=['BestEstimateTurnover'], 
                                     zoom = 8, height=1200, width=1200)
                                    
rev_geog_fig_pane = pn.pane.Plotly(rev_geog_fig)
rev_geog_layout = pn.Column(rev_geog_pane, rev_geog_fig_pane)

In [ ]:

radio_buttons = pn.widgets.RadioButtonGroup(options=['Scot Mfg. Company locations', 'Scot Mfg. Sector maturity'], 
                                                    button_type = 'primary', orientation = 'vertical')

@pn.depends(radio_buttons)
def temp_type(x):
    if x == 'Scot Mfg. Company locations':       
        temp_layout = rev_geog_layout
        return temp_layout
    if x == 'Scot Mfg. Sector maturity':
        temp_layout = pn.Column(db_size_layout,app)
        return temp_layout
        
temp_type = pn.bind(temp_type, x=radio_buttons)
template = pn.template.FastListTemplate(
            site="Dashboard", title="Scot Mfg. Data",
            sidebar = [radio_buttons], background_color = '#6f7271',
            header_background = '#3e4765', neutral_color = '#6f7271',
            accent_base_color = '#3e4765', header_accent_base_color = '#3e4765', 
            main=[temp_type],theme="dark", sidebar_width = 250
            ).servable()
